## Классификатор языков 

In [15]:
import keras
import pandas as pd

РЕШЕНИЕ

Подаю на вход последовательность букв с тегом языка
т.е. допустим, слово ner, то на вход должен войти список [n,e,r] и пометка тега (языка)

С помощью токенайзера подаю секвенсы, их преобразую  в пад сиквенсы
Далее бью на трейн и тест и обучаю на простой модели RNN (ниже составлена)

НИЖЕ БОЛЕЕ ПОДРОБНО

функция для считки из файла


In [16]:
def read(name, maxsize):
  name = open(name, "r")
  arr = []
  i = -1
# считываем все строки
  lines = name.readlines()

# итерация по строкам
  for line in lines:
    
    if i != maxsize:
      i+=1
      #rint(line.strip())
      arr.append(line.strip())
    else:
      continue
# закрываем файл
  name.close
  print(i)
  return arr

In [17]:
languages = ['dutch', 'hungarian', 'portugese']
df = pd.DataFrame(['word'], ['tag'])

In [18]:
def place_tag(number, tag):
  i = 0
  arr = []
  for i in range(number):
    arr.append(tag)
  return arr



In [19]:
df.head()

,0
tag,word


ПЕРЕДЕЛАТЬ, разбить слова на буквы массив, напротив слова поставить тег 

In [20]:
df = pd.DataFrame({"word": [read("hungarian.txt",4555 ), read("dutch.txt",4554 ), read("portugese.txt",4554 )],
                   "tag": [place_tag(4555, 'hungarian'),place_tag(4555, 'dutch'),place_tag(4555, 'portugese')]})

4554
4554
4554


In [21]:
df.head()

,word,tag
0,"[nem, az, hogy, és, egy, van, ez, is, meg, de,...","[hungarian, hungarian, hungarian, hungarian, h..."
1,"[ik, je, het, de, dat, is, een, niet, en, wat,...","[dutch, dutch, dutch, dutch, dutch, dutch, dut..."
2,"[o, de, em, e, que, ser, um, por, para, a, não...","[portugese, portugese, portugese, portugese, p..."


In [22]:
df['word']

0    [nem, az, hogy, és, egy, van, ez, is, meg, de,...
1    [ik, je, het, de, dat, is, een, niet, en, wat,...
2    [o, de, em, e, que, ser, um, por, para, a, não...
Name: word, dtype: object

Каждый в каждом списке список из разбитых слов, ['h', 'o', 'g', 'y'] 

In [30]:
hungarian = []
for i in range(len(df['word'][0])):
  hungarian.append(list(df['word'][0][i]))
dutch = []
for i in range(len(df['word'][1])):
  dutch.append(list(df['word'][1][i]))
portugese = []
for i in range(len(df['word'][2])):
  portugese.append(list(df['word'][2][i]))



In [47]:
hungarian[2]

['h', 'o', 'g', 'y']

Хочу, чтобы каждому списку букв соответствовал тег его языка

|word|-----------------------|tag|

|['n','e','m']| -------------|hungarian|


и тд

не успел создать такой датафрейм

In [44]:
dataf = pd.DataFrame

In [40]:
dataf = pd.DataFrame({'word' :  [hungarian],
                     'tag' : [place_tag(4555, 'hungarian')]})

In [41]:
dataf.head()


,word,tag
0,"[[n, e, m], [a, z], [h, o, g, y], [é, s], [e, ...","[hungarian, hungarian, hungarian, hungarian, h..."


In [33]:
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [36]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words=4554, filters=None)


In [38]:
tokenizer.fit_on_texts(dataf.word)

tokens = tokenizer.texts_to_sequences(dataf.word)
word2idx = tokenizer.word_index
idx2word = tokenizer.index_word


AttributeError: ignored

In [49]:
df = df.groupby("index").agg(list)

In [50]:
tokenizer.fit_on_texts(df.tag)

tags = tokenizer.texts_to_sequences(df.tag)
word2idx_tags = tokenizer.word_index
idx2word_tags = tokenizer.index_word

Разбиваю выборки с помощью склерн

In [51]:
pad_tokens = pad_sequences(tokens, maxlen=40, dtype='int32', padding='post')
pad_tags = pad_sequences(tags, maxlen=40, dtype='int32', padding='post')

In [52]:
pad_tokens

array([[4514, 4515, 4516, 4517, 4518, 4519, 4520, 4521, 4522, 4523, 4524,
        4525, 4526, 4527, 4528, 4529, 4530, 4531, 4532, 4533, 4534, 4535,
        4536, 4537, 4538, 4539, 4540, 4541, 4542, 4543, 4544, 4545, 4546,
        4547, 4548, 4549, 4550, 4551, 4552, 4553],
       [  78,  119,    2,  120,   45,  121,   49,  122,  123,  124,   88,
          76,   58,   92,    1,   35,    8,  125,    2,   20,  126,   94,
          79,  127,  128,   40,  129,    8,   48,    2,    9,  130,  131,
         132,   37,  133,  134,  135,   90,  136],
       [   6,  138,  102,  111,  110,  145,   50,  107,  129,  147,  114,
          97,   95,  148,  122,   68,   32,   62,  149,    1,  140,  149,
         118,   72,   99,  119,  134,  127,  116,  131,   75,  148,  101,
         130,  147,  144,  136,  146,   82,    7]], dtype=int32)

In [54]:
train_tokens, test_tokens, train_tags, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.2, train_size=0.8)

ValueError: ignored

Модель

In [ ]:
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, Dropout
from tensorflow.keras.utils import plot_model

Сама модель  построена на основе SimpleRNN


На выходе 3 класса соответственно языку

использую софтмакс и SparseCategoricalCrossentropy для классификации

In [ ]:

model = Sequential()
model.add(Embedding(input_dim=4554+1, output_dim=64, input_length=40))
model.add(SimpleRNN(16,return_sequences = True))
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation="softmax"))


model.compile(optimizer='adam',loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(train_tokens, train_tags, validation_data=(test_tokens, test_tags), epochs=8, batch_size=128, workers=-1)